In [3]:
# The code was removed by Watson Studio for sharing.

The project consists in clustering the Halifax Regional Municipality (in Nova Scotia, Canada) areas by habitability type according to 6 parameters obtained from ‘’Foursquare’’: distance from an elementary school, distance from a Middle school, distance from a high school, distance from a park, distance from a hockey arena, and distance from a grocery store.

In [4]:
import pandas as pd
import numpy as np

In [5]:
import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
! pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 15.9MB/s eta 0:00:01
Libraries imported.


Setting the limits of the grid

In [6]:
Lat_max=44.833095
Long_min=-63.766778
Lat_min=44.555358
Long_max=-63.419335
Lat_center=(Lat_max+Lat_min)/2
Long_center=(Long_max+Long_min)/2

Building the grid and storing it in a dataframe (Hal_coor).

In [4]:
Hal_coor=pd.DataFrame()
increment=0.01
i=Lat_min

while i<Lat_max:
        j=Long_min
        while j<Long_max:
            a_row = pd.Series([i, j])
            row_df = pd.DataFrame([a_row])
            Hal_coor = pd.concat([row_df, Hal_coor], ignore_index=True)
            j=j+increment
            
        i=i+increment

Hal_coor.head()


,0,1
0,44.825358,-63.426778
1,44.825358,-63.436778
2,44.825358,-63.446778
3,44.825358,-63.456778
4,44.825358,-63.466778


In [5]:
Hal_coor.rename(columns={0:'Latitude', 1:'Longitude'}, inplace=True)
Hal_coor.head()

,Latitude,Longitude
0,44.825358,-63.426778
1,44.825358,-63.436778
2,44.825358,-63.446778
3,44.825358,-63.456778
4,44.825358,-63.466778


The grid starts with 980 coordinates.

In [6]:
Hal_coor.shape

(980, 2)

Preparing to use the Foursquare API.

In [6]:
CLIENT_ID = 'VZ2LSZ35DKAO4COFNM4K1SKRZPFG5OIGGW3LBPZFJ4O1ZAA3' # your Foursquare ID
CLIENT_SECRET = 'LIXMKOHUPJRVIJK1PTJTFWDLW45OXX3DORYDKOOAADEJMISJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VZ2LSZ35DKAO4COFNM4K1SKRZPFG5OIGGW3LBPZFJ4O1ZAA3
CLIENT_SECRET:LIXMKOHUPJRVIJK1PTJTFWDLW45OXX3DORYDKOOAADEJMISJ


Setting the venue categories and making the first request to Foursquare to extract all Elementary schools inside the grid and storing the results in results_Elementary_school.

In [7]:
Elementary_school_cat='4f4533804b9074f6e4fb0105'
High_school_cat='4bf58dd8d48988d13d941735'
Park_cat='4bf58dd8d48988d163941735'
Middle_school_cat='4f4533814b9074f6e4fb0106'
Grocery_store_cat='4bf58dd8d48988d118951735'
Hockey_arena_cat='4bf58dd8d48988d185941735'

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&intent={}&sw={}&ne={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            Elementary_school_cat,
            'browse',
            str(Lat_min)+','+str(Long_min),
            str(Lat_max)+','+str(Long_max),
            )

results_Elementary_school = requests.get(url).json()["response"]
results_Elementary_school

{'venues': [{'id': '509acb11e4b0cce9c3285104',
   'name': 'Astral Drive Elementary School',
   'location': {'lat': 44.656271112423134,
    'lng': -63.47932292918657,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.656271112423134,
      'lng': -63.47932292918657}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'categories': [{'id': '4f4533804b9074f6e4fb0105',
     'name': 'Elementary School',
     'pluralName': 'Elementary Schools',
     'shortName': 'Elementary School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1587809100',
   'hasPerk': False},
  {'id': '50243330e4b0e6fe1a98add9',
   'name': 'Admiral westphal playground',
   'location': {'lat': 44.685343,
    'lng': -63.544957,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.685343,
      'lng': -63.544957}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddr

Putting the elementary school results in the Elementary_school_df dataframe. We get 24 elementary schools

In [20]:


Elementary_school_df=pd.DataFrame(columns=['Name','Latitude','Longitude'])

for v in results_Elementary_school['venues']:
    row=pd.Series([v['name'],v['location']['lat'],v['location']['lng']],index=Elementary_school_df.columns)
    row_df=pd.DataFrame([row])
    Elementary_school_df=pd.concat([row_df,Elementary_school_df],ignore_index=True)   



Elementary_school_df


,Name,Latitude,Longitude
0,Bedford South School,44.707758,-63.679161
1,Westmount Elementary School,44.649978,-63.614802
2,Harrietsfield Elementary,44.568680,-63.634681
3,Humber Park Elementary School,44.704951,-63.494509
4,Admiral Westphal Elementary,44.697536,-63.547099
5,Alderney School,44.678747,-63.546698
6,St. Joseph's-Alexander McKay Elementary,44.662301,-63.596383
7,Prospect Road Elementary,44.564692,-63.730019
8,Hillside Park Elementary School,44.757589,-63.662714
9,École Beaubassin,44.698348,-63.663388


Extracting High schools.

In [8]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&intent={}&sw={}&ne={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            High_school_cat,
            'browse',
            str(Lat_min)+','+str(Long_min),
            str(Lat_max)+','+str(Long_max),
            )

results_High_school = requests.get(url).json()["response"]
results_High_school

{'venues': [{'id': '4ce92794f1c6236ad4ef63f0',
   'name': 'Dartmouth High School',
   'location': {'lat': 44.671230207371345,
    'lng': -63.57273986701076,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.671230207371345,
      'lng': -63.57273986701076}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'categories': [{'id': '4bf58dd8d48988d13d941735',
     'name': 'High School',
     'pluralName': 'High Schools',
     'shortName': 'High School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1587809136',
   'hasPerk': False},
  {'id': '4bcf3e54caff95215dc7ccf0',
   'name': 'Lockview High School',
   'location': {'address': '148 Lockview Road',
    'lat': 44.82315769578471,
    'lng': -63.619379723293385,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.82315769578471,
      'lng': -63.619379723293385}],
    'postalCode': 'B2T 

In [9]:
High_school_df=pd.DataFrame(columns=['Name','Latitude','Longitude'])

for v in results_High_school['venues']:
    row=pd.Series([v['name'],v['location']['lat'],v['location']['lng']],index=High_school_df.columns)
    row_df=pd.DataFrame([row])
    High_school_df=pd.concat([row_df,High_school_df],ignore_index=True)   



High_school_df

,Name,Latitude,Longitude
0,Citadel High School,44.647407,-63.584747
1,Millwood High School,44.790619,-63.708158
2,Ecolé Shannon Park School,44.684231,-63.603465
3,Halifax Christian academy,44.641591,-63.636517
4,JL Ilsley High,44.606930,-63.609598
5,Ecole Sommet,44.700672,-63.678276
6,Flecs,44.709790,-63.740129
7,Charles P Allen High School,44.720431,-63.698116
8,Prince Andrew High School,44.682010,-63.526737
9,Sackville High School,44.775044,-63.674856


In [ ]:
Extracting Middle schools

In [10]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&intent={}&sw={}&ne={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            Middle_school_cat,
            'browse',
            str(Lat_min)+','+str(Long_min),
            str(Lat_max)+','+str(Long_max),
            )

results_Middle_school = requests.get(url).json()["response"]
results_Middle_school

{'venues': [{'id': '4b6ef2d0f964a520acd22ce3',
   'name': 'sackville heights junior high',
   'location': {'lat': 44.7783995778711,
    'lng': -63.70059664544194,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.7783995778711,
      'lng': -63.70059664544194}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'categories': [{'id': '4f4533814b9074f6e4fb0106',
     'name': 'Middle School',
     'pluralName': 'Middle Schools',
     'shortName': 'Middle School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1587809149',
   'hasPerk': False},
  {'id': '50473a55e4b00b4755dd5893',
   'name': 'St Agnes Jr High',
   'location': {'lat': 44.647719,
    'lng': -63.619046,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.647719,
      'lng': -63.619046}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'categ

In [11]:
Middle_school_df=pd.DataFrame(columns=['Name','Latitude','Longitude'])

for v in results_Middle_school['venues']:
    row=pd.Series([v['name'],v['location']['lat'],v['location']['lng']],index=Middle_school_df.columns)
    row_df=pd.DataFrame([row])
    Middle_school_df=pd.concat([row_df,Middle_school_df],ignore_index=True)   



Middle_school_df

,Name,Latitude,Longitude
0,Bicentennial Junior High School,44.671110,-63.571375
1,Caledonia Junior High School,44.686980,-63.530788
2,Graham Creighton Junior High,44.704992,-63.473802
3,Ridgecliff Middle School,44.635666,-63.682460
4,ellenvale junior high school,44.678740,-63.519957
5,Harold T Barrett Junior High,44.822527,-63.683812
6,Eric Graves Memorial Junior High School,44.677589,-63.507610
7,Cunard Junior High,44.621822,-63.610371
8,a.j smeltzer junior high,44.764625,-63.671241
9,John Martin Junior High,44.681810,-63.586286


Extracting Parks. The lone industrial park obtained is deleted.

In [12]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&intent={}&sw={}&ne={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            Park_cat,
            'browse',
            str(Lat_min)+','+str(Long_min),
            str(Lat_max)+','+str(Long_max),
            )

results_Park = requests.get(url).json()["response"]
results_Park

{'venues': [{'id': '51acbf70498ec70486d325a7',
   'name': 'Burnside Industral Park',
   'location': {'lat': 44.69950993072225,
    'lng': -63.589085984139885,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.69950993072225,
      'lng': -63.589085984139885}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1587809160',
   'hasPerk': False},
  {'id': '4b9eb7b4f964a5202dfc36e3',
   'name': 'Tremont Plateau Park',
   'location': {'lat': 44.673745035013326,
    'lng': -63.65572121275553,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.673745035013326,
      'lng': -63.65572121275553}],
    'cc': 'CA',
    'country': 'Canada',
    'formattedAddress': ['Canada'

In [13]:
Park_df=pd.DataFrame(columns=['Name','Latitude','Longitude'])

for v in results_Park['venues']:
    row=pd.Series([v['name'],v['location']['lat'],v['location']['lng']],index=Park_df.columns)
    row_df=pd.DataFrame([row])
    Park_df=pd.concat([row_df,Park_df],ignore_index=True)   



Park_df

,Name,Latitude,Longitude
0,Playground,44.651457,-63.606173
1,Henry Finlay Park,44.669182,-63.563060
2,Park (Dublin at Chebucto),44.649639,-63.599644
3,Sackville Landing,44.767201,-63.680892
4,Ferry Terminal Park,44.663396,-63.568218
5,Shubie Park,44.702239,-63.554160
6,Squiggle Park,44.651314,-63.582410
7,Hemlock Ravine,44.688623,-63.663623
8,Rockingham Elementary School,44.677783,-63.655762
9,Fort Needham,44.664483,-63.600510


In [78]:
Park_df=Park_df[Park_df['Name']!='Burnside Industral Park']
Park_df

,Name,Latitude,Longitude
0,Playground,44.651457,-63.606173
1,Henry Finlay Park,44.669182,-63.563060
2,Park (Dublin at Chebucto),44.649639,-63.599644
3,Sackville Landing,44.767201,-63.680892
4,Ferry Terminal Park,44.663396,-63.568218
5,Shubie Park,44.702239,-63.554160
6,Squiggle Park,44.651314,-63.582410
7,Hemlock Ravine,44.688623,-63.663623
8,Rockingham Elementary School,44.677783,-63.655762
9,Fort Needham,44.664483,-63.600510


Grocery stores are extracted.

In [14]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&intent={}&sw={}&ne={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            Grocery_store_cat,
            'browse',
            str(Lat_min)+','+str(Long_min),
            str(Lat_max)+','+str(Long_max),
            )

results_Grocery_store = requests.get(url).json()["response"]
results_Grocery_store

{'venues': [{'id': '4b4e430ff964a52044e726e3',
   'name': 'Sobeys Clayton Park',
   'location': {'address': '287 Lacewood Dr',
    'crossStreet': 'Willett St',
    'lat': 44.663103,
    'lng': -63.657239,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.663103,
      'lng': -63.657239}],
    'postalCode': 'B3M 3Y7',
    'cc': 'CA',
    'city': 'Halifax',
    'state': 'NS',
    'country': 'Canada',
    'formattedAddress': ['287 Lacewood Dr (Willett St)',
     'Halifax NS B3M 3Y7',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d118951735',
     'name': 'Grocery Store',
     'pluralName': 'Grocery Stores',
     'shortName': 'Grocery Store',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1587809179',
   'hasPerk': False},
  {'id': '4da83e1c1e72d9bb47409812',
   'name': 'Sobeys',
   'location': {'address': '55 Peakview Way',
    'lat': 44.69838116684133,
    'lng

In [15]:
Grocery_store_df=pd.DataFrame(columns=['Name','Latitude','Longitude'])

for v in results_Grocery_store['venues']:
    row=pd.Series([v['name'],v['location']['lat'],v['location']['lng']],index=Grocery_store_df.columns)
    row_df=pd.DataFrame([row])
    Grocery_store_df=pd.concat([row_df,Grocery_store_df],ignore_index=True)   



Grocery_store_df

,Name,Latitude,Longitude
0,Gouthro's No Frills,44.671308,-63.577587
1,Sobeys,44.672300,-63.490246
2,Sobeys Penhorn,44.671606,-63.539322
3,Sobeys Express,44.672828,-63.663317
4,Pete's Frootique,44.732401,-63.655802
5,Sobeys,44.693777,-63.497486
6,Sobeys,44.679778,-63.537851
7,No Frills,44.612641,-63.619475
8,M&M Food Market,44.661600,-63.656724
9,Organic Earth Market,44.644109,-63.602594


Hockey arenas are extracted.

In [16]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&intent={}&sw={}&ne={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            Hockey_arena_cat,
            'browse',
            str(Lat_min)+','+str(Long_min),
            str(Lat_max)+','+str(Long_max),
            )

results_Hockey_arena = requests.get(url).json()["response"]
results_Hockey_arena

{'venues': [{'id': '4ec2ca80a17c78a3a3b81fee',
   'name': 'Halifax Mooseheads',
   'location': {'address': '1800 Argyle Street',
    'lat': 44.648230827620374,
    'lng': -63.57707394919855,
    'labeledLatLngs': [{'label': 'display',
      'lat': 44.648230827620374,
      'lng': -63.57707394919855}],
    'postalCode': 'B3J 3N8',
    'cc': 'CA',
    'city': 'Halifax',
    'state': 'NS',
    'country': 'Canada',
    'formattedAddress': ['1800 Argyle Street',
     'Halifax NS B3J 3N8',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d185941735',
     'name': 'Hockey Arena',
     'pluralName': 'Hockey Arenas',
     'shortName': 'Hockey',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_hockey_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1587809190',
   'hasPerk': False},
  {'id': '4fa5bedee4b0b487ce52f7ad',
   'name': 'Sackville Arena',
   'location': {'lat': 44.767678182529714,
    'lng': -63.66852539583423,
  

In [17]:
Hockey_arena_df=pd.DataFrame(columns=['Name','Latitude','Longitude'])

for v in results_Hockey_arena['venues']:
    row=pd.Series([v['name'],v['location']['lat'],v['location']['lng']],index=Hockey_arena_df.columns)
    row_df=pd.DataFrame([row])
    Hockey_arena_df=pd.concat([row_df,Hockey_arena_df],ignore_index=True)   
    
Hockey_arena_df

,Name,Latitude,Longitude
0,Bowles Arena,44.657407,-63.545055
1,Rocky Lake Dome,44.755344,-63.645205
2,Scotiabank Centre,44.648133,-63.577184
3,Centennial Arena,44.662070,-63.634612
4,Shannon Park Arena,44.685869,-63.601482
5,Gray Arena,44.688738,-63.581900
6,Spryfield Rink,44.608320,-63.608736
7,Halifax Forum,44.655365,-63.604112
8,Shearwater Flyers Forum,44.631739,-63.517582
9,Sackville Arena,44.767678,-63.668525


Using the geodesic function in geopy, we calculate, for each grid point, the distance to the closest elementary school and store it in Hal_coor.

In [18]:
! pip install geopy
from geopy.distance import geodesic

In [21]:


for i in Hal_coor.index:
    for j in Elementary_school_df.index:
        distance=geodesic((Hal_coor.loc[i,'Latitude'],Hal_coor.loc[i,'Longitude']),(Elementary_school_df.loc[j,'Latitude'],Elementary_school_df.loc[j,'Longitude'])).kilometers
        if j==0:
            Hal_coor.loc[i,'Ele distance'] = distance
        else:
            if Hal_coor.loc[i,'Ele distance'] > distance:
                Hal_coor.loc[i,'Ele distance'] = distance
            

Hal_coor.head()

,Latitude,Longitude,Ele distance
0,44.825358,-63.426778,14.414915
1,44.825358,-63.436778,13.902062
2,44.825358,-63.446778,13.165437
3,44.825358,-63.456778,12.435514
4,44.825358,-63.466778,11.713546


In [98]:
Hal_coor

,Latitude,Longitude,Ele distance
0,44.825358,-63.426778,14.414915
1,44.825358,-63.436778,13.902062
2,44.825358,-63.446778,13.165437
3,44.825358,-63.456778,12.435514
4,44.825358,-63.466778,11.713546
5,44.825358,-63.476778,11.001099
6,44.825358,-63.486778,10.300150
7,44.825358,-63.496778,9.613213
8,44.825358,-63.506778,8.943519
9,44.825358,-63.516778,8.295244


Using the geodesic function in geopy, we calculate, for each grid point, the distance to the closest High school and store it in Hal_coor.

In [22]:
for i in Hal_coor.index:
    for j in High_school_df.index:
        distance=geodesic((Hal_coor.loc[i,'Latitude'],Hal_coor.loc[i,'Longitude']),(High_school_df.loc[j,'Latitude'],High_school_df.loc[j,'Longitude'])).kilometers
        if j==0:
            Hal_coor.loc[i,'High distance'] = distance
        else:
            if Hal_coor.loc[i,'High distance'] > distance:
                Hal_coor.loc[i,'High distance'] = distance
            

Hal_coor.head()

,Latitude,Longitude,Ele distance,High distance
0,44.825358,-63.426778,14.414915,15.040899
1,44.825358,-63.436778,13.902062,14.251885
2,44.825358,-63.446778,13.165437,13.463095
3,44.825358,-63.456778,12.435514,12.674570
4,44.825358,-63.466778,11.713546,11.886365


Using the geodesic function in geopy, we calculate, for each grid point, the distance to the closest Middle school and store it in Hal_coor.

In [23]:
for i in Hal_coor.index:
    for j in Middle_school_df.index:
        distance=geodesic((Hal_coor.loc[i,'Latitude'],Hal_coor.loc[i,'Longitude']),(Middle_school_df.loc[j,'Latitude'],Middle_school_df.loc[j,'Longitude'])).kilometers
        if j==0:
            Hal_coor.loc[i,'Middle distance'] = distance
        else:
            if Hal_coor.loc[i,'Middle distance'] > distance:
                Hal_coor.loc[i,'Middle distance'] = distance
            

Hal_coor.head()

,Latitude,Longitude,Ele distance,High distance,Middle distance
0,44.825358,-63.426778,14.414915,15.040899,13.884372
1,44.825358,-63.436778,13.902062,14.251885,13.693357
2,44.825358,-63.446778,13.165437,13.463095,13.545989
3,44.825358,-63.456778,12.435514,12.674570,13.443702
4,44.825358,-63.466778,11.713546,11.886365,13.387529


In [ ]:
Using the geodesic function in geopy, we calculate, for each grid point, the distance to the closest park and store it in Hal_coor.

In [24]:
for i in Hal_coor.index:
    for j in Park_df.index:
        distance=geodesic((Hal_coor.loc[i,'Latitude'],Hal_coor.loc[i,'Longitude']),(Park_df.loc[j,'Latitude'],Park_df.loc[j,'Longitude'])).kilometers
        if j==0:
            Hal_coor.loc[i,'Park distance'] = distance
        else:
            if Hal_coor.loc[i,'Park distance'] > distance:
                Hal_coor.loc[i,'Park distance'] = distance
            

Hal_coor.head()

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance
0,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958
1,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498
2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978
3,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538
4,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394


Using the geodesic function in geopy, we calculate, for each grid point, the distance to the closest Hockey Arena and store it in Hal_coor.

In [25]:
for i in Hal_coor.index:
    for j in Hockey_arena_df.index:
        distance=geodesic((Hal_coor.loc[i,'Latitude'],Hal_coor.loc[i,'Longitude']),(Hockey_arena_df.loc[j,'Latitude'],Hockey_arena_df.loc[j,'Longitude'])).kilometers
        if j==0:
            Hal_coor.loc[i,'Arena distance'] = distance
        else:
            if Hal_coor.loc[i,'Arena distance'] > distance:
                Hal_coor.loc[i,'Arena distance'] = distance
            

Hal_coor.head()

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance
0,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958,18.955320
1,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498,18.236611
2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978,17.524160
3,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538,16.818765
4,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394,16.121351


Using the geodesic function in geopy, we calculate, for each grid point, the distance to the closest Grocery Store and store it in Hal_coor.

In [26]:
for i in Hal_coor.index:
    for j in Grocery_store_df.index:
        distance=geodesic((Hal_coor.loc[i,'Latitude'],Hal_coor.loc[i,'Longitude']),(Grocery_store_df.loc[j,'Latitude'],Grocery_store_df.loc[j,'Longitude'])).kilometers
        if j==0:
            Hal_coor.loc[i,'Grocery distance'] = distance
        else:
            if Hal_coor.loc[i,'Grocery distance'] > distance:
                Hal_coor.loc[i,'Grocery distance'] = distance
            

Hal_coor.head()

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance
0,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958,18.955320,14.537177
1,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498,18.236611,13.747579
2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978,17.524160,12.958141
3,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538,16.818765,12.168894
4,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394,16.121351,11.379878


In [105]:
Hal_coor

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance
0,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958,18.955320,14.537177
1,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498,18.236611,13.747579
2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978,17.524160,12.958141
3,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538,16.818765,12.168894
4,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394,16.121351,11.379878
5,44.825358,-63.476778,11.001099,11.098545,13.378052,14.990810,15.433000,10.591144
6,44.825358,-63.486778,10.300150,10.311201,13.415370,14.685049,14.754980,9.802761
7,44.825358,-63.496778,9.613213,9.524450,13.499093,14.416329,14.088783,9.014820
8,44.825358,-63.506778,8.943519,8.738451,13.628368,14.171738,13.436168,8.227450
9,44.825358,-63.516778,8.295244,7.953429,13.214083,13.442269,12.799213,7.440830


In [27]:
Hal_coor.describe()

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance
count,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000
mean,44.690358,-63.596778,4.122376,4.419859,5.450082,5.657145,6.320254,4.631100
std,0.080819,0.101047,2.555880,2.971115,3.184795,3.553224,3.716402,3.010501
min,44.555358,-63.766778,0.134482,0.097211,0.135847,0.056004,0.124602,0.082487
25%,44.622858,-63.686778,2.203260,2.156127,2.916345,2.768418,3.380980,2.282229
50%,44.690358,-63.596778,3.658886,3.720216,5.118292,5.323409,5.915018,4.111405
75%,44.757858,-63.506778,5.632229,6.105738,7.507471,7.843361,8.567224,6.422954
max,44.825358,-63.426778,14.414915,15.040899,15.021770,16.996958,18.955320,14.537177


We use the On Water API to check for each grid point if it is on water. We then delete the points on water.

In [32]:

import time
Token_water='P2nJpoB6rvnyS_vGh8s9'


for i in Hal_coor.index:
    lat=Hal_coor.loc[i,'Latitude']
    long=Hal_coor.loc[i,'Longitude']
    url='https://api.onwater.io/api/v1/results/{},{}?access_token={}'.format(lat,long,Token_water)
    print(url)
    on_water=requests.get(url).json()['water']
    print(on_water)
    Hal_coor.loc[i,'On water']=on_water
    time.sleep(4.5)
    
Hal_coor.head()


https://api.onwater.io/api/v1/results/44.825357999999945,-63.42677800000007?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.43677800000007?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.446778000000066?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.456778000000064?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.46677800000006?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.47677800000006?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.48677800000006?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.496778000000056?access_token=P2nJpoB6rvnyS_vGh8s9
False
https://api.onwater.io/api/v1/results/44.825357999999945,-63.50677800

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
0,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958,18.955320,14.537177,False
1,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498,18.236611,13.747579,False
2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978,17.524160,12.958141,False
3,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538,16.818765,12.168894,False
4,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394,16.121351,11.379878,False


This process took over an hour since we were limited to 15 requests per minute so the dataframe was saved to a file.

In [35]:
Hal_coor.head()

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
0,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958,18.955320,14.537177,False
1,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498,18.236611,13.747579,False
2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978,17.524160,12.958141,False
3,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538,16.818765,12.168894,False
4,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394,16.121351,11.379878,False


In [37]:

project.save_data(file_name = "Hal_coor.csv",data = Hal_coor.to_csv(index=False))

{'file_name': 'Hal_coor.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'pythondatasciencecapstoneproject-donotdelete-pr-490egpiplsxlv2',
 'asset_id': '9768344e-fb69-475b-8282-d3e522cfbdad'}

In [8]:
my_file = project.get_file("Hal_coor.csv")

# Read the CSV data file from the object storage into a pandas DataFrame
my_file.seek(0)
Hal_coor_df=pd.read_csv(my_file)
Hal_coor_df.head()

,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
0,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958,18.955320,14.537177,False
1,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498,18.236611,13.747579,False
2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978,17.524160,12.958141,False
3,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538,16.818765,12.168894,False
4,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394,16.121351,11.379878,False


In [10]:
Hal_land_df=Hal_coor_df[Hal_coor_df['On water']==False]
Hal_land_df.head()
Hal_land_df.shape

(789, 9)

After eliminating points landing on water, we get 789 grid points, down from the original 980 (see above). Preparing the dataframe for clustering.

In [11]:
Hal_clus_df=Hal_land_df.drop(['Latitude','Longitude','On water'],1)
Hal_clus_df.head()

,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance
0,14.414915,15.040899,13.884372,16.996958,18.955320,14.537177
1,13.902062,14.251885,13.693357,16.539498,18.236611,13.747579
2,13.165437,13.463095,13.545989,16.107978,17.524160,12.958141
3,12.435514,12.674570,13.443702,15.704538,16.818765,12.168894
4,11.713546,11.886365,13.387529,15.331394,16.121351,11.379878


Using six clusters.

In [12]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(Hal_clus_df)
kmeans


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

Adding the cluster labels to the dataframe.

In [13]:
Hal_land_df.insert(0, 'Cluster Labels', kmeans.labels_)
Hal_land_df.head()

,Cluster Labels,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
0,2,44.825358,-63.426778,14.414915,15.040899,13.884372,16.996958,18.955320,14.537177,False
1,2,44.825358,-63.436778,13.902062,14.251885,13.693357,16.539498,18.236611,13.747579,False
2,2,44.825358,-63.446778,13.165437,13.463095,13.545989,16.107978,17.524160,12.958141,False
3,2,44.825358,-63.456778,12.435514,12.674570,13.443702,15.704538,16.818765,12.168894,False
4,2,44.825358,-63.466778,11.713546,11.886365,13.387529,15.331394,16.121351,11.379878,False


Grouping by cluster and averaging.

In [14]:
Hal_grouped_df=Hal_land_df.groupby('Cluster Labels').mean().reset_index()
Hal_grouped_df

,Cluster Labels,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
0,0,44.716267,-63.672163,6.283272,4.451900,5.026434,6.713402,8.532396,4.613670,False
1,1,44.685625,-63.607634,2.063156,1.743353,2.136671,1.643846,2.361176,1.504853,False
2,2,44.805358,-63.462632,10.595728,11.448286,11.365682,13.734887,15.351645,11.122676,False
3,3,44.701176,-63.616733,3.456565,2.985688,4.144961,4.054377,4.816078,3.231454,False
4,4,44.657453,-63.565159,3.127706,5.295799,7.367124,6.986884,6.211304,6.175125,False
5,5,44.706218,-63.593875,5.401273,8.600565,8.437119,10.296077,11.321052,7.293213,False


In [15]:
Hal_grouped_df

,Cluster Labels,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
0,0,44.716267,-63.672163,6.283272,4.451900,5.026434,6.713402,8.532396,4.613670,False
1,1,44.685625,-63.607634,2.063156,1.743353,2.136671,1.643846,2.361176,1.504853,False
2,2,44.805358,-63.462632,10.595728,11.448286,11.365682,13.734887,15.351645,11.122676,False
3,3,44.701176,-63.616733,3.456565,2.985688,4.144961,4.054377,4.816078,3.231454,False
4,4,44.657453,-63.565159,3.127706,5.295799,7.367124,6.986884,6.211304,6.175125,False
5,5,44.706218,-63.593875,5.401273,8.600565,8.437119,10.296077,11.321052,7.293213,False


Displaying the grid points on the map. The different colors each represent a cluster.

In [26]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#ys = [i + x + (i*x)**2 for i in range(6)]

Hal_map = folium.Map(location=[Lat_center, Long_center], zoom_start=11)
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

palette=['black','red','blue','yellow','green','violet']

for i in Hal_land_df.index:
    lat=Hal_land_df.loc[i,'Latitude']
    long=Hal_land_df.loc[i,'Longitude']
    label=Hal_land_df.loc[i,'Cluster Labels']
    folium.CircleMarker(
        [lat, long],
        radius=2,
        popup=label,
        color=palette[label],
        fill=True,
        fill_color=palette[label],
        fill_opacity=0.7).add_to(Hal_map)

Hal_map

Focussing only on cluster 1 and re-clustering to get a better view of the residential areas.

In [27]:
zoom_Hal_df=Hal_land_df[Hal_land_df['Cluster Labels']==1]
zoom_Hal_df.head()

,Cluster Labels,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
128,1,44.795358,-63.656778,4.223379,2.672675,2.065335,2.371368,3.213434,2.121955,False
129,1,44.795358,-63.666778,4.209471,2.346271,1.666091,2.063607,3.079097,1.874542,False
130,1,44.795358,-63.676778,4.342279,2.262618,1.593308,2.036215,3.144571,1.943631,False
131,1,44.795358,-63.686778,4.609139,1.771881,1.885186,2.299233,3.398328,2.300885,False
132,1,44.795358,-63.696778,4.988584,1.043187,1.908625,2.771170,3.802861,2.488296,False


In [28]:
zoom_Hal_df.shape

(187, 10)

In [29]:
zoom_Hal_clus=zoom_Hal_df.drop(['Cluster Labels','Latitude','Longitude','On water'],1)
zoom_Hal_clus.head()

,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance
128,4.223379,2.672675,2.065335,2.371368,3.213434,2.121955
129,4.209471,2.346271,1.666091,2.063607,3.079097,1.874542
130,4.342279,2.262618,1.593308,2.036215,3.144571,1.943631
131,4.609139,1.771881,1.885186,2.299233,3.398328,2.300885
132,4.988584,1.043187,1.908625,2.771170,3.802861,2.488296


In [30]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(zoom_Hal_clus)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [32]:
zoom_Hal_df=zoom_Hal_df.drop('Cluster Labels',1)
zoom_Hal_df.insert(0, 'Cluster Labels', kmeans.labels_)
zoom_Hal_df.head()

,Cluster Labels,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
128,0,44.795358,-63.656778,4.223379,2.672675,2.065335,2.371368,3.213434,2.121955,False
129,0,44.795358,-63.666778,4.209471,2.346271,1.666091,2.063607,3.079097,1.874542,False
130,0,44.795358,-63.676778,4.342279,2.262618,1.593308,2.036215,3.144571,1.943631,False
131,0,44.795358,-63.686778,4.609139,1.771881,1.885186,2.299233,3.398328,2.300885,False
132,0,44.795358,-63.696778,4.988584,1.043187,1.908625,2.771170,3.802861,2.488296,False


Grouping by clusters and averaging.

In [33]:
zoom_grouped_df=zoom_Hal_df.groupby('Cluster Labels').mean().reset_index()
zoom_grouped_df

,Cluster Labels,Latitude,Longitude,Ele distance,High distance,Middle distance,Park distance,Arena distance,Grocery distance,On water
0,0,44.696570,-63.655263,3.677567,1.785703,2.128934,2.222938,2.946603,2.094143,False
1,1,44.681858,-63.500778,1.202474,1.271257,1.737972,2.692041,4.625681,1.338022,False
2,2,44.668758,-63.593978,1.055187,1.325994,1.248956,0.960678,1.638033,0.897793,False
3,3,44.689733,-63.665528,1.494219,1.875915,4.731648,0.873138,3.774061,1.098078,False
4,4,44.675929,-63.617921,2.670802,1.351397,1.897273,1.236666,1.303123,1.357402,False
5,5,44.710813,-63.606475,2.128971,2.970919,2.726800,2.270120,1.936133,2.290071,False


Displaying the new clusters on a map.

In [35]:
zoom_Hal_map = folium.Map(location=[Lat_center, Long_center], zoom_start=11)
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

palette=['black','red','blue','yellow','green','violet']

for i in zoom_Hal_df.index:
    lat=zoom_Hal_df.loc[i,'Latitude']
    long=zoom_Hal_df.loc[i,'Longitude']
    label=zoom_Hal_df.loc[i,'Cluster Labels']
    folium.CircleMarker(
        [lat, long],
        radius=2,
        popup=label,
        color=palette[label],
        fill=True,
        fill_color=palette[label],
        fill_opacity=0.7).add_to(zoom_Hal_map)

zoom_Hal_map